<a href="https://colab.research.google.com/github/amarjeet-cheema/CNN-application/blob/main/Sentiment_analysis_using_LSTM_Movie_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd


In [ ]:
tf.__version__

'2.4.0'

In [ ]:
data= pd.read_csv('/IMDB Dataset.csv')


In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df=data.copy()

In [ ]:
df.dropna
df.reset_index(inplace=True)

In [ ]:
X=df['review']
X.head()

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. <br /><br />The...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
Name: review, dtype: object

In [ ]:
y=df['sentiment']
y.head()

0    positive
1    positive
2    positive
3    negative
4    positive
Name: sentiment, dtype: object

In [ ]:
df['review'][6]

"I sure would like to see a resurrection of a up dated Seahunt series with the tech they have today it would bring back the kid excitement in me.I grew up on black and white TV and Seahunt with Gunsmoke were my hero's every week.You have my vote for a comeback of a new sea hunt.We need a change of pace in TV and this would work for a world of under water adventure.Oh by the way thank you for an outlet like this to view many viewpoints about TV and the many movies.So any ole way I believe I've got what I wanna say.Would be nice to read some more plus points about sea hunt.If my rhymes would be 10 lines would you let me submit,or leave me out to be in doubt and have me to quit,If this is so then I must go so lets do it."

In [ ]:
df.head()

,index,review,sentiment
0,0,One of the other reviewers has mentioned that ...,positive
1,1,A wonderful little production. <br /><br />The...,positive
2,2,I thought this was a wonderful way to spend ti...,positive
3,3,Basically there's a family where a little boy ...,negative
4,4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
len(df)

50000

In [ ]:
import nltk
nltk.download('stopwords') 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps = PorterStemmer()
corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['review'][i]) # substitute all the characters other than a-zA-Z to a blank in each message
    review = review.lower()
    review = review.split() # splits the sentences into individual list of words
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #Apply stemmer for all the words not in stopwoprd list
    review = ' '.join(review) # Add the stemmed word to the orginal list with spaces in between instead of ''
    corpus.append(review) # Add the word to the complete list of words i.e corpus
    

In [ ]:
len(corpus)

50000

In [ ]:
corpus[0]

'one review mention watch oz episod hook right exactli happen br br first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word br br call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away br br would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch da

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

In [ ]:
#Returns index of the words in corpus
onehot_repr=[one_hot(words,voc_size)for words in corpus] 


In [ ]:
le=[]
for i in range(len(corpus)):
  new = len(corpus[i])
  le.append(new)

In [ ]:
sent_length=max(le)
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1199 3262 4214]
 [   0    0    0 ... 1427 1137 4254]
 [   0    0    0 ... 2108 3994 4339]
 ...
 [   0    0    0 ...  726  302 2973]
 [   0    0    0 ... 4926 2583 4084]
 [   0    0    0 ... 1047 4091 2589]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0, ..., 1199, 3262, 4214], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 8350, 40)          200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(50000, (50000,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((50000, 8350), (50000,))

In [ ]:
y_final

array(['positive', 'positive', 'positive', ..., 'negative', 'negative',
       'negative'], dtype=object)

In [ ]:
y_new= pd.get_dummies(y_final)


,negative,positive
0,0,1
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
49995,0,1
49996,1,0
49997,1,0
49998,1,0


In [ ]:
y_label=y_new['positive'].to_numpy()
y_label

array([1, 1, 1, ..., 0, 0, 0], dtype=uint8)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_label, test_size=0.33, random_state=42)

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
524/524 [==============================] - 233s 443ms/step - loss: 0.5274 - accuracy: 0.7242 - val_loss: 0.3258 - val_accuracy: 0.8633
Epoch 2/10
524/524 [==============================] - 231s 441ms/step - loss: 0.3128 - accuracy: 0.8728 - val_loss: 0.3236 - val_accuracy: 0.8590
Epoch 3/10
524/524 [==============================] - 231s 440ms/step - loss: 0.2734 - accuracy: 0.8922 - val_loss: 0.3430 - val_accuracy: 0.8587
Epoch 4/10
524/524 [==============================] - 233s 444ms/step - loss: 0.2551 - accuracy: 0.9004 - val_loss: 0.3368 - val_accuracy: 0.8587
Epoch 5/10
524/524 [==============================] - 232s 444ms/step - loss: 0.2397 - accuracy: 0.9067 - val_loss: 0.3653 - val_accuracy: 0.8608
Epoch 6/10
524/524 [==============================] - 233s 444ms/step - loss: 0.2165 - accuracy: 0.9160 - val_loss: 0.3535 - val_accuracy: 0.8604
Epoch 7/10
524/524 [==============================] - 232s 442ms/step - loss: 0.2027 - accuracy: 0.9211 - val_loss: 0.4798 -

In [ ]:
y_pred=model.predict_classes(X_test)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
confusion_matrix(y_test,y_pred)


array([[7028, 1180],
       [1197, 7095]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8559393939393939